# COMPASS Engine: Interactive Demo

## Clinical Orchestrated Multi-modal Predictive Agentic Support System

This notebook shows how to run COMPASS on a participant folder and inspect outputs.

What you'll do:
- Launch the web dashboard (optional)
- Run the pipeline in-notebook
- Inspect the final report + execution log

Prerequisites:
- Data under `data/pseudo_data` (demo) or `data/__FEATURES__/COMPASS_data`
- If using OpenAI backend, set `OPENAI_API_KEY` in your environment


In [ ]:
import sys
import json
import ipywidgets as widgets
from IPython.display import display, Markdown, JSON, clear_output
from pathlib import Path
import subprocess
import time

# --- PATH SETUP ---
# We are in /utils. Navigate up two levels to project root (INFERENCE_PIPELINE)
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"[*] Project Root: {project_root}")

from multi_agent_system.main import run_compass_pipeline
from multi_agent_system.config.settings import get_settings, LLMBackend

# Locate Data
PSEUDO_DATA_ROOT = project_root / "multi_agent_system" / "data" / "pseudo_data"
if not PSEUDO_DATA_ROOT.exists():
    # Fallback to main data dir if pseudo_data is missing
    PSEUDO_DATA_ROOT = project_root / "data" / "__FEATURES__" / "COMPASS_data"

print(f"✅ COMPASS Engine Loaded. Data Source: {PSEUDO_DATA_ROOT.name}")

## Quick Launch: Web Dashboard

Launch the live UI (`main.py --ui`) for step-by-step monitoring in your browser.
This uses the same Python environment as the notebook.


In [ ]:
def launch_dashboard(b):
    clear_output()
    print("🚀 Launching Dashboard on port 5005...")
    print("Go to: http://127.0.0.1:5005")
    try:
        # Use sys.executable to ensure we use the same python env
        main_script = project_root / "multi_agent_system" / "main.py"
        subprocess.Popen([sys.executable, str(main_script), "--ui"])
    except Exception as e:
        print(f"Error launching: {e}")

btn = widgets.Button(description="Launch Web Dashboard", button_style='info', icon='rocket')
btn.on_click(launch_dashboard)
display(btn)

## Interactive Notebook Execution

Run the pipeline directly in the notebook (CLI mode).
This is useful for quick iteration and debugging without the web dashboard.


### 1. Configuration
Select a subject, target condition, backend, and number of iterations.
If you choose the Local backend, ensure the model is available on this machine.


In [ ]:
# Find available subjects
subjects = [d.name for d in PSEUDO_DATA_ROOT.iterdir() if d.is_dir() and not d.name.startswith('.')]
subjects.sort()

# Interactive Widgets
subject_dropdown = widgets.Dropdown(
    options=subjects,
    description='Subject:',
)

target_input = widgets.Text(
    value='Major Depressive Disorder',
    description='Target Pattern:',
    placeholder='e.g. Anxiety'
)

iterations_input = widgets.BoundedIntText(
    value=2,
    min=1,
    max=5,
    description='Iterations:',
)

# Backend Selection
backend_dropdown = widgets.Dropdown(
    options=[('OpenAI API (GPT-5)', 'openai'), ('Local LLM (Open Source)', 'local')],
    value='openai',
    description='Backend:',
)

model_input = widgets.Text(
    value='Qwen/Qwen2.5-0.5B-Instruct',
    description='Local Model:',
    disabled=True
)

def on_backend_change(change):
    if change['new'] == 'local':
        model_input.disabled = False
    else:
        model_input.disabled = True

backend_dropdown.observe(on_backend_change, names='value')

display(subject_dropdown, target_input, iterations_input, backend_dropdown, model_input)


### 2. Run Analysis
This executes the pipeline and prints verbose output so you can follow agent reasoning.


In [ ]:
PARTICIPANT_DIR = PSEUDO_DATA_ROOT / subject_dropdown.value
TARGET = target_input.value
MAX_ITERS = iterations_input.value

print(f"Selected Participant: {PARTICIPANT_DIR}")

# Apply Backend Settings
settings = get_settings()
if backend_dropdown.value == 'local':
    settings.models.backend = LLMBackend.LOCAL
    settings.models.local_model_name = model_input.value
    print(f"Configured for LOCAL inference using {model_input.value}")
else:
    settings.models.backend = LLMBackend.OPENAI
    print("Configured for OPENAI inference")

print(f"Starting analysis for: {TARGET}...")

# Run Pipeline
try:
    result = run_compass_pipeline(
        participant_dir=PARTICIPANT_DIR,
        target_condition=TARGET,
        max_iterations=MAX_ITERS,
        verbose=True,
        interactive_ui=False  # Use CLI output mode for notebook compatibility
    )
except Exception as e:
    print(f"Execution Error: {e}")
    result = None


### 3. Inspector
Review the final report and the structured execution log generated by the run.


In [ ]:
if result:
    log_path = Path(result['output_dir']) / f"execution_log_{result['participant_id']}.json"
    report_path = Path(result['output_dir']) / f"report_{result['participant_id']}.md"

    print(f"\n📊 RESULT: {result['prediction']} ({result['probability']:.1%})")
    
    if report_path.exists():
        display(Markdown("### Final Clinical Report"))
        with open(report_path, 'r') as f:
            display(Markdown(f.read()))
else:
    print("No successful result to inspect.")